# <b><u>Store Sales - Time Series Forecasting <b><u> 

`This is a time series forecasting problem. In this project, you'll predict store sales on data from Corporation Favorita, a large Ecuadorian-based grocery retailer`

- `we are to build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores.`

- `The training data includes dates, store, and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models`

## <B> Description<B>

train.csv

- `The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.`

test.csv

- `The test data, having the same features as the training data. You will predict the target sales for the dates in this file.`
- `The dates in the test data are for the 15 days after the last date in the training data.`

transaction.csv

- `Contains date, store_nbr and transaction made on that specific date.`

sample_submission.csv

- `A sample submission file in the correct format.`

stores.csv

- `Store metadata, including city, state, type, and cluster.`

oil.csv

- `Daily oil price which includes values during both the train and test data timeframes.`

holidays_events.csv

- `Holidays and Events, with metadata`

## <b>Hypothesis & Questions

The questions below are to be answered. Do note that, you are free to draw more hypothesis from the data.

Is the train dataset complete (has all the required dates)?

Which dates have the lowest and highest sales for each year?

Did the earthquake impact sales?

Are certain groups of stores selling more products? (Cluster, city, state, type)

Are sales affected by promotions, oil prices and holidays?

What analysis can we get from the date and its extractable features?

What is the difference between RMSLE, RMSE, MSE (or why is the MAE greater than all of them?)

## Importation

In [2]:
#data manupulation packages
import numpy as np 
import  pandas as pd 
#data visualization package
import seaborn as sn 
import matplotlib as plt 
#data connection package
import pyodbc
from dotenv import dotenv_values
#ignore warnings(optional)
import warnings
warnings.filterwarnings('ignore')
#machine learning packages
from sklearn.pipeline import Pipeline



## <b>Data importation

### <i><b> load SQL database; table 1, table 2 and table 3

In [3]:
#load the environment variable
environment_variable= dotenv_values('.env')
#get the values for the credentials from the .env file
server_name= environment_variable.get('SERVER_NAME')
user = environment_variable.get('USER')
password= environment_variable.get('PASSWORD')
database_name= environment_variable.get('DATABASE_NAME')
#a connection to the server, user, password and database name
connection_string= f"DRIVER={{SQL server}};SERVER={server_name};DATABASE_NAME={database_name};UID={user};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"



In [4]:
#connectin to the server
connection= pyodbc.connect(connection_string)

In [6]:
# Now the sql query to get the data is what what you see below. 
# Note that you will not have permissions to insert delete or update this database table. 

query = "SELECT * FROM dbo.oil"

oil_table = pd.read_sql(query, connection)

#Holiday_events table 

query = "SELECT * FROM dbo.holidays_events"

holiday_events = pd.read_sql(query, connection)

#Stores Table

query = "SELECT * FROM dbo.stores"

stores_table = pd.read_sql(query, connection)

DatabaseError: Execution failed on sql: SELECT * FROM dbo.oil
('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')
unable to rollback

### Load the csv dataset

In [ ]:

# Load the third dataset (it's a CSV file named 'test', 'transaction', 'sample_submission')

test_data = pd.read_csv('test.csv')

transaction_data = pd.read_csv('transactions.csv')

sample_data = pd.read_csv('sample_submission.csv')